In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re

## Чтение данных

In [2]:
df_amo= pd.read_csv('amocrm_sample_export.csv', encoding='utf-8')

In [3]:
df_values= pd.read_csv('list_of_values.csv', encoding='utf-8')

In [4]:
df_sample= pd.read_csv('certificates_list_sample.csv', encoding='utf-8')

In [5]:
df_archive= pd.read_excel('codes_archieve.xlsx')

In [6]:
#Вывод таблицы во всю ширину
pd.set_option('display.max_columns', None)
df_amo.head(5)

,ID,Этап сделки,Бюджет,Дата создания сделки,Дата закрытия,DELIVERY_PRICE,Стоимость доставки,PRODUCTS,PRICE,SUBTOTAL,Выручка от клиента,Направление (вид сертификата),Номер сертификата,КОРНЕВАЯ СДЕЛКА,Номер 2-го сертификата,Сертификат на фотографа,Сертификат на тренера,Статус погашения,Дата оплаты подрядчику
0,26165303,Успешно реализовано,5500,30.11.2023 17:43:20,30.11.2023 18:00:00,0.0,NaN,катание на картинге (Варианты: 40 минут катани...,5500.0,5500.0,5500,Картинг,VKA4_4314611,NaN,NaN,NaN,NaN,NaN,NaN
1,26163311,Успешно реализовано,8000,30.11.2023 14:58:27,01.12.2023 15:09:00,NaN,NaN,Мотокросс и Эндуро (Варианты: Персональная тре...,8000.0,8000.0,8000,Мото Мотокросс,MT 273176,NaN,NaN,NaN,NaN,NaN,NaN
2,26162989,Успешно реализовано,5450,30.11.2023 14:33:51,01.12.2023 15:20:00,450.0,NaN,Стрельба из боевого оружия (Варианты: Курс «на...,5450.0,5000.0,5450,Тир. Стрельба из боевого оружия,GU2 21041,NaN,NaN,NaN,NaN,NaN,NaN
3,26160427,Успешно реализовано,34567,30.11.2023 11:37:23,02.12.2023 16:01:00,NaN,NaN,"Универсальный 23456 (Варианты: 34 567 руб., Бу...",34567.0,34567.0,34567,Универс. 23.456 и 34.567,WTF3 269703,NaN,NaN,NaN,NaN,NaN,NaN
4,26158865,Успешно реализовано,9000,30.11.2023 09:39:04,30.11.2023 09:47:00,NaN,NaN,Аэротруба (Варианты: 10 минут полёта (для 1-2 ...,9000.0,9000.0,9000,Аэротруба,VAT10 4353286,NaN,NaN,NaN,NaN,NaN,NaN


## Предобработка архивных префиксов

In [7]:
df_archive.head()

,Направление,Номинал,Примечание,Префикс,Префикс VIRT,Ячейки,Сезонность,Цены на сайте,Закупка
0,Водные лыжи,водные лыжи за катером 1 час,NaN,SKI60,VSKI60,1,ЛЕТО \n05 - 10,8500,7600
1,Водные лыжи,водные лыжи за катером 40 минут,NaN,SKI40,VSKI40,2,ЛЕТО \n05 - 10,6000,5400
2,Водные лыжи,водные лыжи за катером 20 минут,NaN,SKI20,VSKI20,3,ЛЕТО \n05 - 10,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Вейксёрф и вейк за катером,вейк и сёрф за катером 1 час,NaN,WS60,VWS60,4,ЛЕТО \n05 - 10,8800,7920


In [8]:
df_archive['Префикс'] = df_archive['Префикс'].str.upper()

In [9]:
df_archive = df_archive.dropna(subset=['Направление','Префикс'])

In [10]:
df_archive = df_archive.reset_index(drop=True)

In [11]:
df_archive.rename(columns = {'Префикс':'prefix'}, inplace = True )

In [12]:
df_archive.head()

,Направление,Номинал,Примечание,prefix,Префикс VIRT,Ячейки,Сезонность,Цены на сайте,Закупка
0,Водные лыжи,водные лыжи за катером 1 час,NaN,SKI60,VSKI60,1,ЛЕТО \n05 - 10,8500,7600
1,Водные лыжи,водные лыжи за катером 40 минут,NaN,SKI40,VSKI40,2,ЛЕТО \n05 - 10,6000,5400
2,Водные лыжи,водные лыжи за катером 20 минут,NaN,SKI20,VSKI20,3,ЛЕТО \n05 - 10,NaN,NaN
3,Вейксёрф и вейк за катером,вейк и сёрф за катером 1 час,NaN,WS60,VWS60,4,ЛЕТО \n05 - 10,8800,7920
4,Вейксёрф и вейк за катером,вейк и сёрф за катером 40 минут,NaN,WS40,VWS40,5,ЛЕТО \n05 - 10,6400,5760


In [13]:
print(sorted(list(df_archive['prefix'].unique())))

['2WP24', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AG15', 'AG20', 'AG30', 'AH', 'AH2', 'AL', 'AP15', 'AP20', 'AP30', 'ARB', 'ARH', 'ART', 'ARV', 'AS15', 'AS20', 'AS30', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K / AT6K', 'AT6', 'AT60', 'BC1', 'BC2', 'BCR', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1+ КАРТА ТРЕНЕР', 'CW2', 'CW2+ КАРТА ТРЕНЕР', 'CWH', 'DC', 'DG15', 'DG20', 'DG30', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1X', 'EB2X', 'EB4X', 'ED3H', 'EDC', 'EF30', 'EF60', 'ES130', 'ES160', 'ES230', 'ES260', 'ESG1', 'ESG2', 'ESG4', 'FJ30', 'FM30 OLD', 'FM45 OLD', 'FM60 OLD', 'FMCS30 OLD', 'FMCS45 OLD', 'FMCS45R OLD', 'FMCS60 OLD', 'FMP15 OLD', 'FMP30 OLD', 'FMS30 OLD', 'FMS45 OLD', 'FMS60 OLD', 'FR', 'FR2', 'FRK', 'FROG1', 'FROG3', 'FROGM', 'FRT', 'FRTK', 'GC1', 'GC2', 'GL20', 'GL40', 'GL60', 'GR1', 'GR1 + T', 'GR2', 'GR2 + T', 'GU', 'GU2', 'HB20', 'HB30', 'HB60', 'HY30', 'HY60', 'ICE1', '

При сортировке обнаружилась кириллица в префиксах

In [14]:
alf_ciril = 'Е Т О Р А Н К Х С В М'.split()
alf_latin = 'E T O P A H K X C B M'.split()

In [15]:
def replace_cirillic_latin(x):
    v = x
    new_v = ''
    for i in range(len(v)):
        simv = v[i]
        if simv in alf_ciril:
            pos = alf_ciril.index(simv)
            simv = alf_latin[pos]
        new_v += simv
    return new_v

In [16]:
df_archive['prefix'] = df_archive.apply(lambda x: replace_cirillic_latin(x['prefix']),axis=1)

Удаление OLD из префикса

In [17]:
df1 = df_archive[df_archive['prefix'].str.contains('OLD')]

In [18]:
df1.head()

,Направление,Номинал,Примечание,prefix,Префикс VIRT,Ячейки,Сезонность,Цены на сайте,Закупка
242,Автоспорт,OLD Контр-аварийная подготовка курс (2 дня по ...,NaN,SDC OLD,VSDC OLD,NaN,NaN,17000,13600
243,Автоспорт,OLD Контр-аварийная подготовка индивидуальное ...,NaN,SDCI OLD,VSDCI OLD,NaN,NaN,33000,???
244,САП-сёрф,OLD Сапборды на 3 часа ТОЛЬКО КАРТОЧКИ,NaN,SU3 OLD,VSU3 OLD,???,NaN,-,-
245,Самолёты,OLD STOL 30 минут для 1 человека,30 минут полёта на самолёте Внедорожник,FMS30 OLD,VFMS30 OLD,NaN,NaN,6000,4800
246,Самолёты,OLD STOL 45 минут для 1 человека,45 минут полёта на самолёте Внедорожник,FMS45 OLD,VFMS45 OLD,NaN,NaN,9000,7200


In [19]:
df1.shape

(33, 9)

In [20]:
df_archive.replace(r' (OLD)|(OLD) ',r'',regex=True,inplace=True)

In [21]:
df1.head()

,Направление,Номинал,Примечание,prefix,Префикс VIRT,Ячейки,Сезонность,Цены на сайте,Закупка
242,Автоспорт,OLD Контр-аварийная подготовка курс (2 дня по ...,NaN,SDC OLD,VSDC OLD,NaN,NaN,17000,13600
243,Автоспорт,OLD Контр-аварийная подготовка индивидуальное ...,NaN,SDCI OLD,VSDCI OLD,NaN,NaN,33000,???
244,САП-сёрф,OLD Сапборды на 3 часа ТОЛЬКО КАРТОЧКИ,NaN,SU3 OLD,VSU3 OLD,???,NaN,-,-
245,Самолёты,OLD STOL 30 минут для 1 человека,30 минут полёта на самолёте Внедорожник,FMS30 OLD,VFMS30 OLD,NaN,NaN,6000,4800
246,Самолёты,OLD STOL 45 минут для 1 человека,45 минут полёта на самолёте Внедорожник,FMS45 OLD,VFMS45 OLD,NaN,NaN,9000,7200


In [22]:
df_archive.iloc[246]['prefix']

'FMS45'

In [23]:
df1.shape

(33, 9)

Удаление **+ Карта Тренер**, **+T**

In [24]:
df_archive.replace(r'( *\+ *\S* *[Т|T]\S*)',r'T',regex=True,inplace=True)

In [25]:
print(sorted(list(df_archive['prefix'].unique())))

['2SU24', '2SU60', '2WP24', 'AA', 'AA2', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AG15', 'AG20', 'AG30', 'AH', 'AH2', 'AL', 'AP15', 'AP20', 'AP30', 'ARB', 'ARH', 'ART', 'ARV', 'AS15', 'AS20', 'AS30', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K / AT6K', 'AT6', 'AT60', 'BC1', 'BC2', 'BCR', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1T', 'CW2', 'CW2T', 'CWH', 'DC', 'DG15', 'DG20', 'DG30', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1X', 'EB2X', 'EB4X', 'ED3H', 'EDC', 'EF30', 'EF60', 'ES130', 'ES160', 'ES230', 'ES260', 'ESG1', 'ESG2', 'ESG4', 'FJ30', 'FM30', 'FM45', 'FM60', 'FMCS30', 'FMCS45', 'FMCS45R', 'FMCS60', 'FMP15', 'FMP30', 'FMS30', 'FMS45', 'FMS60', 'FR', 'FR2', 'FRK', 'FROG1', 'FROG3', 'FROGM', 'FRT', 'FRTK', 'GC1', 'GC2', 'GL20', 'GL40', 'GL60', 'GR1', 'GR1T', 'GR2', 'GR2T', 'GU', 'GU2', 'HB20', 'HB30', 'HB60', 'HY30', 'HY60', 'ICE1', 'ICE1M', 'ICE2', 'ID', 'JT2', 'JT2A', 'JT3', 'JT3A

Имеются двойные префиксы MT/MT2 и MT10/MT10B

In [26]:
archive_prefix = list(df_archive['prefix'].unique())

## Предобработка активных префиксов

In [27]:
df_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              198 non-null    int64  
 1   Activity Name   198 non-null    object 
 2   Value           196 non-null    object 
 3   Status          198 non-null    object 
 4   Prefix          198 non-null    object 
 5   Purchase Price  167 non-null    float64
 6   Sale Price      196 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 11.0+ KB


In [28]:
df_values.columns = df_values.columns.str.lower()

In [29]:
df_values.columns = df_values.columns.str.strip()

In [30]:
df_values.columns = df_values.columns.map(lambda x: re.sub( '(?<!^)(?=[ ])', '_', x ).lower())
df_values.columns = df_values.columns.map(lambda x: re.sub( ' ', '', x ))

In [31]:
df_values.head(5)

,id,activity_name,value,status,prefix,purchase_price,sale_price
0,1,Вейксёрф и вейкборд,2 часа вейксерфинга или вейкбординга\nна волне...,on_air,WS2,NaN,22500.0
1,2,Вейксёрф и вейкборд,1 час вейксерфинга или вейкбординга\nна волне ...,on_air,WS1,NaN,11000.0
2,3,Вейксёрф и вейкборд,40 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS40,NaN,7500.0
3,4,Вейксёрф и вейкборд,30 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS30,NaN,5500.0
4,5,Вейксёрф и вейкборд,25 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS25,NaN,5000.0


In [32]:
df_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              198 non-null    int64  
 1   activity_name   198 non-null    object 
 2   value           196 non-null    object 
 3   status          198 non-null    object 
 4   prefix          198 non-null    object 
 5   purchase_price  167 non-null    float64
 6   sale_price      196 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 11.0+ KB


In [33]:
print(sorted(list(df_values['prefix'].unique())))

['2MWP24', '2WP24', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K', 'AT6', 'AT60', 'AT6K', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1T', 'CW2', 'CW2T', 'CWH', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1x', 'EB2x', 'EB4x', 'ED3H', 'EDС', 'EFL30', 'EFL60', 'FR', 'FR2,', 'FRK', 'FRT', 'FRTK', 'GC1', 'GC2', 'GR1', 'GR2', 'GU', 'GU2', 'HB30', 'HB60', 'HY30', 'HY60', 'KA2', 'KA4', 'KA6', 'KAT', 'KAT2', 'KAT4', 'KS', 'KS2', 'KV1X1', 'KV2X1', 'KVP1X1', 'KVP2X1', 'KW', 'KW2', 'KYT', 'KYT2', 'LL1H', 'LL1HP', 'LL2H', 'LL2HP', 'LL3H', 'MB', 'MB2', 'MB2X', 'MEFL30', 'MEFL60', 'MF20', 'MF30', 'MF60', 'MHB60', 'MHY30', 'MHY60', 'MMF20', 'MMF30', 'MMF60', 'MSKI40', 'MSKI60', 'MSU24', 'MSU60', 'MT10', 'MW&F', 'MWP15', 'MWP2H', 'MWP60', 'MWS1', 'MWS2', 'MWS25', 'MWS30', 'MWS40', 'MWSN1', 'MWSN2', 'MWSN25', 'MWSN40', 'OW2', 'OWG', 'OWI', 'PL15', 'PL20', 'PL30', 'PP', '

Все prefix прописные. Имеется кириллица в префиксах

In [34]:
df_values['prefix'] = df_values['prefix'].str.upper()

In [35]:
df_values['prefix'] = df_values.apply(lambda x: replace_cirillic_latin(x['prefix']),axis=1)

In [36]:
prefix_values = list(df_values['prefix'].unique())

In [37]:
print(sorted((prefix_values)))

['2MWP24', '2WP24', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K', 'AT6', 'AT60', 'AT6K', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1T', 'CW2', 'CW2T', 'CWH', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1X', 'EB2X', 'EB4X', 'ED3H', 'EDC', 'EFL30', 'EFL60', 'FR', 'FR2,', 'FRK', 'FRT', 'FRTK', 'GC1', 'GC2', 'GR1', 'GR2', 'GU', 'GU2', 'HB30', 'HB60', 'HY30', 'HY60', 'KA2', 'KA4', 'KA6', 'KAT', 'KAT2', 'KAT4', 'KS', 'KS2', 'KV1X1', 'KV2X1', 'KVP1X1', 'KVP2X1', 'KW', 'KW2', 'KYT', 'KYT2', 'LL1H', 'LL1HP', 'LL2H', 'LL2HP', 'LL3H', 'MB', 'MB2', 'MB2X', 'MEFL30', 'MEFL60', 'MF20', 'MF30', 'MF60', 'MHB60', 'MHY30', 'MHY60', 'MMF20', 'MMF30', 'MMF60', 'MSKI40', 'MSKI60', 'MSU24', 'MSU60', 'MT', 'MT10', 'MT5', 'MTT', 'MW&F', 'MWP15', 'MWP2H', 'MWP60', 'MWS1', 'MWS2', 'MWS25', 'MWS30', 'MWS40', 'MWSN1', 'MWSN2', 'MWSN25', 'MWSN40', 'OW2', 'OWG', 'OWI', 'PB', 'PB2'

In [38]:
len(prefix_values)

197

#### Объединенный список префиксов

In [39]:
all_prefix = list(set(prefix_values).union(set(archive_prefix)))

In [40]:
all_prefix.sort()

In [41]:
print(all_prefix)

['2MWP24', '2SU24', '2SU60', '2WP24', 'AA', 'AA2', 'AB', 'AB4', 'AB4G', 'ABG', 'AC2', 'AC30', 'AC60', 'AG15', 'AG20', 'AG30', 'AH', 'AH2', 'AL', 'AP15', 'AP20', 'AP30', 'ARB', 'ARH', 'ART', 'ARV', 'AS15', 'AS20', 'AS30', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K', 'AT4K / AT6K', 'AT6', 'AT60', 'AT6K', 'BC1', 'BC2', 'BCR', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CSR60', 'CW1', 'CW1T', 'CW2', 'CW2T', 'CWH', 'DC', 'DG15', 'DG20', 'DG30', 'DRC2H', 'DRCH', 'DT30', 'DT32', 'DT40', 'DT42', 'DT60', 'DT62', 'DTM', 'EB1X', 'EB2X', 'EB4X', 'ED3H', 'EDC', 'EF30', 'EF60', 'EFL30', 'EFL60', 'ES130', 'ES160', 'ES230', 'ES260', 'ESG1', 'ESG2', 'ESG4', 'FJ30', 'FM30', 'FM45', 'FM60', 'FMCS30', 'FMCS45', 'FMCS45R', 'FMCS60', 'FMP15', 'FMP30', 'FMS30', 'FMS45', 'FMS60', 'FR', 'FR2', 'FR2,', 'FRK', 'FROG1', 'FROG3', 'FROGM', 'FRT', 'FRTK', 'GC1', 'GC2', 'GL20', 'GL40', 'GL60', 'GR1', 'GR1T', 'GR2', 'GR2T', 'GU', 'GU2', 'HB20', 'HB30', 'HB60', 'HY30', 'HY60', 'ICE1'

In [42]:
len(all_prefix)

308

In [43]:
archive_prefix = list(set(archive_prefix) - set(prefix_values))

In [44]:
archive_prefix.sort()

Измененный **archive_prefix**

In [45]:
print(archive_prefix)

['2SU24', '2SU60', 'AA', 'AA2', 'AG15', 'AG20', 'AG30', 'AH', 'AH2', 'AL', 'AP15', 'AP20', 'AP30', 'ARB', 'ARH', 'ART', 'ARV', 'AS15', 'AS20', 'AS30', 'AT4K / AT6K', 'BC1', 'BC2', 'BCR', 'DC', 'DG15', 'DG20', 'DG30', 'EF30', 'EF60', 'ES130', 'ES160', 'ES230', 'ES260', 'ESG1', 'ESG2', 'ESG4', 'FJ30', 'FM30', 'FM45', 'FM60', 'FMCS30', 'FMCS45', 'FMCS45R', 'FMCS60', 'FMP15', 'FMP30', 'FMS30', 'FMS45', 'FMS60', 'FR2', 'FROG1', 'FROG3', 'FROGM', 'GL20', 'GL40', 'GL60', 'GR1T', 'GR2T', 'HB20', 'ICE1', 'ICE1M', 'ICE2', 'ID', 'JT2', 'JT2A', 'JT3', 'JT3A', 'JT4', 'JT4A', 'JT7', 'JT7A', 'KR2', 'KR4', 'KR6', 'KVS', 'LB1X', 'LB2X', 'LB4X', 'MT/MT2', 'MT10/MT10B', 'MT10B', 'MT2B', 'MT5/MT5B', 'MT5B', 'PH', 'RS', 'SB', 'SB4', 'SBG', 'SDC', 'SDCI', 'SG7', 'SKI20', 'SN1X1', 'SN2X1', 'SU3', 'SU3H', 'TR', 'TR1', 'TR1K', 'TR2', 'TR2K', 'UC', 'UN', 'VD', 'WP20', 'WP30', 'WS20', 'WS60', 'WSN60']


In [46]:
len(archive_prefix)

111

In [47]:
df_amo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2622 entries, 0 to 2621
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ID                             2622 non-null   int64  
 1   Этап сделки                    2622 non-null   object 
 2   Бюджет                         2622 non-null   int64  
 3   Дата создания сделки           2622 non-null   object 
 4   Дата закрытия                  2622 non-null   object 
 5   DELIVERY_PRICE                 1647 non-null   float64
 6   Стоимость доставки             9 non-null      float64
 7   PRODUCTS                       2452 non-null   object 
 8   PRICE                          2452 non-null   float64
 9   SUBTOTAL                       2452 non-null   float64
 10  Выручка от клиента             2498 non-null   object 
 11  Направление (вид сертификата)  2622 non-null   object 
 12  Номер сертификата              2622 non-null   o

In [48]:
df_values.head(10)

,id,activity_name,value,status,prefix,purchase_price,sale_price
0,1,Вейксёрф и вейкборд,2 часа вейксерфинга или вейкбординга\nна волне...,on_air,WS2,NaN,22500.0
1,2,Вейксёрф и вейкборд,1 час вейксерфинга или вейкбординга\nна волне ...,on_air,WS1,NaN,11000.0
2,3,Вейксёрф и вейкборд,40 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS40,NaN,7500.0
3,4,Вейксёрф и вейкборд,30 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS30,NaN,5500.0
4,5,Вейксёрф и вейкборд,25 минут вейксерфинга или вейкбординга\nна вол...,on_air,WS25,NaN,5000.0
5,6,Вейксёрф и вейкборд,2 часа вейксерфинга или вейкбординга\nна любой...,on_air,WSN2,NaN,26500.0
6,7,Вейксёрф и вейкборд,1 час вейксерфинга или вейкбординга на любой в...,on_air,WSN1,NaN,13500.0
7,8,Вейксёрф и вейкборд,40 минут вейксерфинга или вейкбординга на любо...,on_air,WSN40,NaN,8500.0
8,9,Вейксёрф и вейкборд,25 минут вейксерфинга или вейкбординга на любо...,on_air,WSN25,NaN,6000.0
9,10,Флайборд,1 час полёта на флайборде,on_air,MF60,NaN,12500.0


## Предобработка базы AMO

### Имена столбцов

Столбцы в нижний регистр и удаление пробелов

In [49]:
df_amo.columns = df_amo.columns.str.lower()

In [50]:
df_amo.columns = df_amo.columns.str.strip()

In [51]:
df_amo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2622 entries, 0 to 2621
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             2622 non-null   int64  
 1   этап сделки                    2622 non-null   object 
 2   бюджет                         2622 non-null   int64  
 3   дата создания сделки           2622 non-null   object 
 4   дата закрытия                  2622 non-null   object 
 5   delivery_price                 1647 non-null   float64
 6   стоимость доставки             9 non-null      float64
 7   products                       2452 non-null   object 
 8   price                          2452 non-null   float64
 9   subtotal                       2452 non-null   float64
 10  выручка от клиента             2498 non-null   object 
 11  направление (вид сертификата)  2622 non-null   object 
 12  номер сертификата              2622 non-null   o

Удаление пробелов и вставка символа "_"

In [52]:
df_amo.columns = df_amo.columns.map(lambda x: re.sub( '(?<!^)(?=[ ])', '_', x ).lower())
df_amo.columns = df_amo.columns.map(lambda x: re.sub( ' ', '', x ))

Копирование исходной базы

In [53]:
df_amo_temp = df_amo.copy()

Переименование столбцов

In [54]:
df_amo_temp.rename(columns = {'номер_сертификата':'code', 'id':'amo_id','дата_закрытия':'sale_date','дата_оплаты_подрядчику':'card_used_date'}, inplace = True )

In [55]:
df_values.rename(columns = {'id':'activity_value_id' }, inplace = True )

In [56]:
df_amo_temp['code'] = df_amo_temp['code'].str.upper()

In [57]:
df_amo_temp['номер_2-го_сертификата'] = df_amo_temp['номер_2-го_сертификата'].str.upper()

In [58]:
df_amo_temp['сертификат_на_фотографа'] = df_amo_temp['сертификат_на_фотографа'].str.upper()

In [59]:
df_amo_temp['сертификат_на_тренера'] = df_amo_temp['сертификат_на_тренера'].str.upper()

In [60]:
df_amo_temp['дата_создания_сделки'] = pd.to_datetime(df_amo_temp['дата_создания_сделки'] )

In [61]:
df_amo_temp['sale_date'] = pd.to_datetime(df_amo_temp['sale_date'])

In [62]:
df_amo_temp['card_used_date'] = pd.to_datetime(df_amo_temp['card_used_date'],errors='coerce')

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '25.11.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '18.11.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '26.11.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '27.10.2023' in DD/MM/YYYY format. Provide format or specif

In [63]:
df_amo_temp = df_amo_temp.sort_values(by=["дата_создания_сделки"], ascending=True)

In [64]:
#df_amo_temp['supplier_payment_date'] = df_amo_temp['card_used_date']

In [65]:
df_amo_temp = df_amo_temp.reset_index(drop=True)

In [66]:
df_amo_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2622 entries, 0 to 2621
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   amo_id                         2622 non-null   int64         
 1   этап_сделки                    2622 non-null   object        
 2   бюджет                         2622 non-null   int64         
 3   дата_создания_сделки           2622 non-null   datetime64[ns]
 4   sale_date                      2622 non-null   datetime64[ns]
 5   delivery_price                 1647 non-null   float64       
 6   стоимость_доставки             9 non-null      float64       
 7   products                       2452 non-null   object        
 8   price                          2452 non-null   float64       
 9   subtotal                       2452 non-null   float64       
 10  выручка_от_клиента             2498 non-null   object        
 11  направление_(вид_

In [67]:
df_amo_temp.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date
0,23580385,Успешно реализовано,5200,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание на картинге (Варианты: 40 минут катани...,5200.0,5200.0,5200,Картинг,KA4 213421,NaN,NaN,NaN,NaN,NaN,NaT
1,23594083,Успешно реализовано,4850,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы (Варианты: 1 квадрик 1 час без пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1 125315,NaN,NaN,NaN,NaN,NaN,NaT
2,23596587,Успешно реализовано,4850,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет на параплане (Варианты: 1 полет с инстру...,4850.0,4500.0,4850,Параплан,P1G 259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09
3,23596239,Успешно реализовано,6000,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк за катером (Варианты: 30 минут Mastercraf...,6000.0,6000.0,6000,Вейксёрф и вейк за катером,WS30 239328,NaN,NaN,NaN,TR1 210177,NaN,NaT
4,23598009,Успешно реализовано,5200,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание на картинге (Варианты: 40 минут катани...,5200.0,5200.0,5200,Картинг,КА4213697,NaN,NaN,NaN,NaN,NaN,NaT


Определение разделителей в поле **code**

In [68]:
cc = df_amo_temp['code'].unique()

In [69]:
set_cc = set()

In [70]:
for el in cc:
    set_cc.update(el)

In [71]:
print(set_cc)

{'H', 'V', '7', ')', 'T', 'К', 'М', 'F', 'D', ' ', 'J', 'L', 'X', 'K', 'Р', 'Т', 'B', '3', 'U', '0', 'P', 'R', 'M', '\xa0', 'N', '8', '(', 'C', 'Н', 'A', '4', 'С', '2', 'А', 'G', '5', '_', 'E', '6', '1', '&', 'S', 'W', 'I', 'И', 'Y', '9', 'В'}


В качестве разделителей используются символы ' ', '_', '\xa0' и их комбинации

В значениях поля code имеются символы кириллицы

Замена кириллицы на латинские символы

In [72]:
#df_amo_temp['code'] = df_amo_temp.apply(replace_cirillic_latin,axis=1)
df_amo_temp['code'] = df_amo_temp.apply(lambda x: replace_cirillic_latin(x['code']),axis=1)

Замена всех возможных разделителей на '_'

In [73]:
#df_amo_temp['code'] = df_amo_temp['code'].str.replace('[^_ \xa0]+([ _\xa0]+)\d+','_',regex=True)
df_amo_temp.replace(r'([ _\xa0]+)',r'_',regex=True,inplace=True)

In [74]:
df_amo_temp.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date
0,23580385,Успешно_реализовано,5200,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT
1,23594083,Успешно_реализовано,4850,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT
2,23596587,Успешно_реализовано,4850,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09
3,23596239,Успешно_реализовано,6000,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT
4,23598009,Успешно_реализовано,5200,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4213697,NaN,NaN,NaN,NaN,NaN,NaT


## Создание отдельных строк для 2-го сертификата

In [75]:
df2sert = df_amo_temp.loc[(df_amo_temp['номер_2-го_сертификата'].notna())] 

In [76]:
df2sert.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date
23,24441731,Успешно_реализовано,8000,2023-01-06 11:32:36,2023-02-06 00:01:00,NaN,NaN,"Самолёт_Cessna_182_(Варианты:_20_минут_полёта,...",8000.0,8000.0,8000,Авиа_Самолёты,CS20_1905810537,NaN,VKA6_362697,NaN,NaN,Погашен,2023-07-21
63,23460055,Успешно_реализовано,10000,2023-01-13 11:50:38,2023-01-13 11:59:00,NaN,NaN,Заезд_на_багги_(Варианты:_Занятие_для_одного_1...,10000.0,10000.0,10000,Мото_Дрифт-трайк,VDTM_3651601,NaN,VBGH_2415955DIS,NaN,NaN,NaN,NaT
75,23476291,Успешно_реализовано,10000,2023-01-16 14:34:03,2023-01-16 16:17:00,NaN,NaN,"Снегоходы_(Варианты:_1_снежик_на_2_часа,_Бумаж...",9000.0,9000.0,9000,Снегоходы,VKVP2X1_2696914,NaN,VSN2X_2416042DIS,NaN,NaN,NaN,NaT
80,23499379,Успешно_реализовано,13750,2023-01-18 15:21:40,2023-01-18 18:43:00,0.0,NaN,Заезд_на_багги_(Варианты:_Занятие_для_детей_1_...,13750.0,13750.0,13750,Багги,VBGH_2430880,NaN,VBGKH_2416119,NaN,NaN,Погашен,2023-05-26
148,23990017,Успешно_реализовано,6000,2023-02-04 10:59:02,2023-04-04 16:25:00,0.0,NaN,"Катер_без_капитана_(Варианты:_1_час_проката,_Б...",5500.0,5500.0,6000,Катер_без_капитана_(Летолодка),VLL1H_173317,NaN,VLL1H_2818826_DIS,NaN,NaN,Погашен,2023-06-18


In [77]:
df2sert.shape

(82, 19)

Имеются 2-е сертификаты, имеющие пометку DIS

Исключение подобных 2-х сертификатов

In [78]:
df2sert = df2sert.loc[~df2sert['номер_2-го_сертификата'].str.contains('DI')]

In [79]:
df2sert.shape

(49, 19)

In [80]:
df2sert.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date
23,24441731,Успешно_реализовано,8000,2023-01-06 11:32:36,2023-02-06 00:01:00,NaN,NaN,"Самолёт_Cessna_182_(Варианты:_20_минут_полёта,...",8000.0,8000.0,8000,Авиа_Самолёты,CS20_1905810537,NaN,VKA6_362697,NaN,NaN,Погашен,2023-07-21
80,23499379,Успешно_реализовано,13750,2023-01-18 15:21:40,2023-01-18 18:43:00,0.0,NaN,Заезд_на_багги_(Варианты:_Занятие_для_детей_1_...,13750.0,13750.0,13750,Багги,VBGH_2430880,NaN,VBGKH_2416119,NaN,NaN,Погашен,2023-05-26
233,23680799,Успешно_реализовано,12500,2023-02-15 11:03:35,2023-02-15 10:37:00,NaN,NaN,Искусственная_волна_Flow_Moscow_(Варианты:_1_ч...,12500.0,12500.0,12500,Искусственная_волна,VCW1T_2557338,NaN,VCW1T_2516846,NaN,NaN,Погашен,2023-03-17
294,23719721,Успешно_реализовано,6000,2023-02-21 16:09:36,2023-02-22 12:03:00,NaN,NaN,Подарочный_сертификатна_вейксёрфинг_(Выберите_...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS40_210129289,NaN,VWS40_2517352,NaN,NaN,Погашен,2023-07-22
409,23778849,Успешно_реализовано,6500,2023-03-03 18:55:38,2023-03-03 19:04:00,NaN,NaN,Искусственная_волна_Flow_Moscow_(Варианты:_1_ч...,6500.0,6500.0,6500,Искусственная_волна,CW1_269598,NaN,TR1_212055,NaN,NaN,Погашен,2023-04-17


In [81]:
def insert_row_sert2(row):
    df_amo_temp.loc[len(df_amo_temp.index)] = [row['amo_id'],
                                           np.NaN,
                                          np.NaN,
                                           row['дата_создания_сделки'],
                                           row['sale_date'],
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          '',
                                          row['номер_2-го_сертификата'],
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN,
                                          np.NaN]
    return 1
    

In [82]:
df_amo_temp.shape

(2622, 19)

In [83]:
df2sert['qq']= df2sert.apply(insert_row_sert2,axis=1)

In [84]:
df_amo_temp.shape

(2671, 19)

In [85]:
df_amo_temp.query('amo_id == 24441731')

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date
23,24441731,Успешно_реализовано,8000.0,2023-01-06 11:32:36,2023-02-06 00:01:00,NaN,NaN,"Самолёт_Cessna_182_(Варианты:_20_минут_полёта,...",8000.0,8000.0,8000,Авиа_Самолёты,CS20_1905810537,NaN,VKA6_362697,NaN,NaN,Погашен,2023-07-21
2622,24441731,NaN,NaN,2023-01-06 11:32:36,2023-02-06 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,,VKA6_362697,NaN,NaN,NaN,NaN,NaN,NaT


В базу AMO добавлены строки для вторых сертификатов

### Формирование новых столбцов в базе АМО

In [86]:
df_amo_temp['id'] = ''

#### prefix and number

In [87]:
df_amo_temp['prefix'] = df_amo_temp['code'].apply(lambda x: re.split('[ _]',x)[0] if ' ' in x or '_' in x else '')

In [88]:
df_amo_temp['number'] = df_amo_temp['code'].apply(lambda x: re.split('[ _]',x)[1] if ' ' in x or '_' in x else '')

In [89]:
df_amo_temp.head(10)

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number
0,23580385,Успешно_реализовано,5200.0,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213421
1,23594083,Успешно_реализовано,4850.0,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,125315
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395
3,23596239,Успешно_реализовано,6000.0,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT,,WS30,239328
4,23598009,Успешно_реализовано,5200.0,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4213697,NaN,NaN,NaN,NaN,NaN,NaT,,,
5,23766769,Успешно_реализовано,6350.0,2023-01-03 12:02:45,2023-02-03 20:30:00,350.0,NaN,Заезд_на_Дрифт_Трайке_(Варианты:_Заезд_40_мину...,6350.0,6000.0,6350,Мото_Дрифт-трайк,DT40_285464,NaN,NaN,NaN,NaN,NaN,NaT,,DT40,285464
6,23695437,Успешно_реализовано,10000.0,2023-01-03 14:55:47,2023-02-17 15:03:00,0.0,NaN,Фридайвинг_(Варианты:_Индивидуальная_тренировк...,10000.0,10000.0,10000,Фридайвинг,VFR2655363,NaN,NaN,NaN,NaN,NaN,NaT,,,
7,23770797,Успешно_реализовано,0.0,2023-01-03 16:05:22,2023-01-03 16:06:00,NaN,NaN,NaN,NaN,NaN,NaN,Фридайвинг,VFR2666164,#23769895,NaN,NaN,NaN,Погашен,2023-10-07,,,
8,23772175,Успешно_реализовано,9350.0,2023-01-03 17:59:38,2023-04-03 17:35:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,9350.0,9000.0,9350,Квадроциклы,KV1X1_162569,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,162569
9,23774335,Успешно_реализовано,6000.0,2023-01-03 22:16:49,2023-02-03 10:20:00,0.0,NaN,"Бигфуты_(Варианты:_40_мин._для_1_с_пассажиром,...",6000.0,6000.0,6000,Квадроциклы_Бигфуты,VBF40_2683331,NaN,NaN,NaN,NaN,NaN,NaT,,VBF40,2683331


In [90]:
prefix_amo = list(df_amo_temp['prefix'].unique())
prefix_amo.sort()
prefix_amo = prefix_amo[1:]

In [91]:
print(prefix_amo)

["'+", 'AC60', 'AT10', 'AT15', 'AT4', 'AT4K', 'AT6', 'AT6K', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'CS20', 'CS30', 'CS40', 'CS60', 'CW1', 'CW2', 'DRC2H', 'DT230', 'DT240', 'DT30', 'DT40', 'DT60', 'EB1X', 'EB2X', 'ED3H', 'EDC', 'EF30', 'EFL30', 'EFL60', 'ES130', 'F1', 'GC1', 'GC2', 'GR1', 'GR1X1', 'GR2', 'GU', 'GU2', 'HB20', 'HB30', 'KA2', 'KA4', 'KA6', 'KAT', 'KAT2', 'KR2', 'KR4', 'KS', 'KS2', 'KV1X1', 'KV2X1', 'KVP1X1', 'KVP2X1', 'LL1H', 'LL2H', 'MF', 'MF20', 'MF30', 'MF60', 'MT', 'MT2', 'MT5', 'MTT', 'P1', 'P1G', 'P1P', 'P2', 'P2G', 'PB', 'PB2', 'PB2X', 'PPP', 'PR20', 'PR30', 'PR40', 'PR60', 'PS', 'RB15', 'RB20', 'RB30', 'RC', 'RCV', 'RJ', 'RJ2', 'RTV', 'RXV', 'S', 'S2', 'SC', 'SC7', 'SG7', 'SH', 'SH2', 'SHH', 'SKI40', 'SKIN60', 'SKT1', 'SKT2', 'SKV', 'SN1X', 'SN1X1', 'SN2X', 'SN2X1', 'SP', 'SP2', 'ST', 'SU24', 'SU60', 'TR', 'TR1', 'TR2', 'TR2K', 'UN', 'VAC30', 'VAC60', 'VAT10', 'VAT15', 'VAT30', 'VAT4', 'VAT4K', 'VAT6', 'VAT6K', 'VBF180', 'VBF40', 'VBF90', 'VBG', 'VBGH', 'VBGK', 'VB

Некоторые значения в code без разделителей, потому не выделились prefix и number

Отделение prefix от number

In [92]:
def separate_prefix_number(row):
    pl = all_prefix
    
    v = row['code']
    if row['prefix'] == '':
        amo_id = row['amo_id']
        
        for el in pl:
            if el in row['code']:
                try:
                    v = el + '_' + row['code'][len(el):] 
                except:
                    pass
                finally:
                    return v
        
        pl = prefix_amo        
        for el in all_prefix:
            if el in row['code']:
                try:
                    v = el + '_' + row['code'][len(el):] 
                except:
                    pass
                finally:
                    return v
    return v

In [93]:
df_amo_temp['code'] = df_amo_temp.apply(separate_prefix_number,axis=1)

In [94]:
df_amo_temp.head(5)

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number
0,23580385,Успешно_реализовано,5200.0,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213421
1,23594083,Успешно_реализовано,4850.0,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,125315
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395
3,23596239,Успешно_реализовано,6000.0,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT,,WS30,239328
4,23598009,Успешно_реализовано,5200.0,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213697,NaN,NaN,NaN,NaN,NaN,NaT,,,


In [95]:
df_amo_temp.loc[df_amo_temp['prefix'] == '','prefix'] = \
df_amo_temp['code'].apply(lambda x: re.split('[_]',x)[0] if ' ' in x or '_' in x else '')

In [96]:
df_amo_temp.loc[df_amo_temp['number'] == '','number'] = \
df_amo_temp['code'].apply(lambda x: re.split('[_]',x)[1] if ' ' in x or '_' in x else '')

In [97]:
df_amo_temp.head(5)

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number
0,23580385,Успешно_реализовано,5200.0,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213421
1,23594083,Успешно_реализовано,4850.0,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,125315
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395
3,23596239,Успешно_реализовано,6000.0,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT,,WS30,239328
4,23598009,Успешно_реализовано,5200.0,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213697,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213697


In [98]:
df_not_separate_prefix = df_amo_temp.loc[~df_amo_temp['code'].str.contains('_')]

In [99]:
df_not_separate_prefix.shape

(32, 22)

In [100]:
df_not_separate_prefix

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number
81,23500055,Успешно_реализовано,21800.0,2023-01-18 16:44:39,2023-01-18 18:43:00,NaN,NaN,Прыжок_с_парашютом_(Варианты:_Прыжок_с_инструк...,21800.0,21800.0,21800,Парашюты,SP267323,NaN,NaN,NaN,NaN,Погашен,2023-01-25,,,
85,23503361,Успешно_реализовано,12795.0,2023-01-19 10:45:43,2023-01-19 10:59:00,450.0,NaN,Универсальный_12345_(Бумажный_или_электронный:...,12795.0,12345.0,12795,Универсальный_Мосвейк_12345,WF284959,NaN,NaN,NaN,NaN,NaN,NaT,,,
105,23547341,Успешно_реализовано,22250.0,2023-01-25 00:23:05,2023-01-25 17:57:00,450.0,NaN,Прыжок_с_парашютом_(Варианты:_Прыжок_с_инструк...,22250.0,21800.0,22500,Парашюты,SP291000,NaN,NaN,NaN,NaN,NaN,NaT,,,
223,23672219,Успешно_реализовано,4500.0,2023-02-13 18:39:27,2023-02-14 09:45:00,NaN,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4500.0,4500.0,4500,Параплан,P1G238229,NaN,NaN,NaN,NaN,NaN,NaT,,,
239,23691835,Успешно_реализовано,5000.0,2023-02-16 20:33:34,2023-02-17 23:16:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,5000,Параплан,80688,NaN,NaN,NaN,NaN,NaN,NaT,,,
495,23871179,Успешно_реализовано,18000.0,2023-03-15 19:21:50,2023-03-15 19:56:00,NaN,NaN,Прыжок_с_парашютом_(Варианты:_Прыжок_с_инструк...,18000.0,18000.0,18000,Парашюты,S223242,NaN,NaN,NaN,NaN,NaN,NaT,,,
532,23925027,Успешно_реализовано,12345.0,2023-03-23 13:03:59,2023-03-25 18:52:00,NaN,NaN,Подарочный_сертификат_на_12_345_рублей_-_1x123...,12345.0,12345.0,12345,Универсальный_Мосвейк_12345,WF258319,NaN,NaN,NaN,NaN,Погашен,2023-08-15,,,
554,23959571,Успешно_реализовано,5600.0,2023-03-28 19:29:43,2023-03-28 22:01:00,NaN,NaN,Аэротруба_(Варианты:_6_минут_полёта_(для_1-2_ч...,5600.0,5600.0,5600,Аэротруба,VAT37112,NaN,NaN,NaN,NaN,NaN,NaT,,,
555,23962927,Успешно_реализовано,6000.0,2023-03-29 11:58:03,2023-03-29 12:03:00,NaN,NaN,Подарочный_сертификатна_сап-сёрфинг_(Выберите_...,6000.0,6000.0,6000,САП-сёрф,VSU215131,NaN,NaN,NaN,NaN,Погашен,2023-05-08,,,
556,23963847,Успешно_реализовано,6000.0,2023-03-29 12:47:10,2023-03-29 13:05:00,0.0,NaN,Подарочный_сертификатв_вейк-парк_(Выберите_тип...,6000.0,6000.0,6000,Вейк-парк,VWP138740,NaN,NaN,NaN,NaN,Погашен,2023-08-18,,,


In [101]:
df_amo_temp['code'] = df_amo_temp.apply(lambda x: re.sub(r'(W)(F)', r'\1&\2', x['code']),axis=1)

#### Дополнительное поле prefix_V для "чистых префиксов" (без V)

In [102]:
df_amo_temp['prefix'] = df_amo_temp.apply(lambda x: replace_cirillic_latin(x['prefix']),axis=1)

In [103]:
df_amo_temp['prefix_V'] = df_amo_temp['prefix']

In [104]:
df_amo_temp.loc[(df_amo_temp['prefix'].str[0] == 'V'),'prefix_V'] = df_amo_temp['prefix'].str[1:]

In [105]:
df_amo_temp.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,prefix_V
0,23580385,Успешно_реализовано,5200.0,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213421,KA4
1,23594083,Успешно_реализовано,4850.0,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,125315,KV1X1
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395,P1G
3,23596239,Успешно_реализовано,6000.0,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT,,WS30,239328,WS30
4,23598009,Успешно_реализовано,5200.0,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213697,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213697,KA4


#### Отсутствующие префиксы

In [106]:
prefix_amo = list(df_amo_temp['prefix_V'].unique())
prefix_amo.sort()
#prefix_amo = prefix_amo[1:]

##### Префиксы в AMO

In [107]:
print(prefix_amo)

['', "'+", 'AC30', 'AC60', 'AT10', 'AT15', 'AT30', 'AT4', 'AT4K', 'AT6', 'AT6K', 'BF180', 'BF40', 'BF90', 'BG', 'BGH', 'BGK', 'BGKH', 'CS20', 'CS30', 'CS40', 'CS60', 'CW1', 'CW1T', 'CW2', 'CW2T', 'DRC2H', 'DT230', 'DT240', 'DT30', 'DT40', 'DT60', 'DT62', 'DTM', 'EB1X', 'EB2X', 'ED3H', 'EDC', 'EF30', 'EF60', 'EFL', 'EFL30', 'EFL60', 'ES130', 'F1', 'FR', 'FR2', 'GC1', 'GC2', 'GR1', 'GR1X1', 'GR2', 'GU', 'GU2', 'HB20', 'HB30', 'HY30', 'HY30T', 'HY60', 'HY60T30', 'KA2', 'KA4', 'KA6', 'KAT', 'KAT2', 'KAT4', 'KR2', 'KR4', 'KS', 'KS2', 'KV1X', 'KV1X1', 'KV2X1', 'KVP1X', 'KVP1X1', 'KVP2X1', 'LB1X', 'LL1H', 'LL2H', 'LL3H', 'MB', 'MB2', 'MB2X', 'MF', 'MF20', 'MF30', 'MF60', 'MT', 'MT2', 'MT5', 'MTT', 'P1', 'P1G', 'P1P', 'P2', 'P2G', 'PB', 'PB2', 'PB2X', 'PP', 'PP2', 'PP2P', 'PPP', 'PPS', 'PR20', 'PR30', 'PR40', 'PR60', 'PS', 'RB15', 'RB20', 'RB30', 'RC', 'RCV', 'RJ', 'RJ1X', 'RJ2', 'RJ2X', 'RTV', 'RXV', 'S', 'S2', 'SC', 'SC10', 'SC7', 'SG7', 'SH', 'SH2', 'SHH', 'SK', 'SK2', 'SK2V', 'SKI40', 'SKI

##### Префиксы, отсутсвующие в общем списке

In [108]:
#missing_prefix = set(prefix_value).difference(set(prefix_amo))
missing_prefix = list(set(prefix_amo)-(set(all_prefix)))
missing_prefix.sort()

In [109]:
print(missing_prefix)

['', "'+", 'DT230', 'DT240', 'EFL', 'F1', 'GR1X1', 'HY30T', 'HY60T30', 'KV1X', 'KVP1X', 'MF', 'MT2', 'P1', 'P1G', 'P1P', 'P2', 'P2G', 'PS', 'RJ1X', 'RJ2X', 'S', 'S2', 'SC', 'SC10', 'SC7', 'SKIN60', 'SP', 'SP2', 'ST', 'SVH', 'WF', 'WPA', 'WS20T', 'WS30T', 'WS40T', 'WS60T', 'WSN40T', 'WSN60T', 'WTF', 'WVS30']


In [110]:
len(missing_prefix)

41

#### Товары, префиксы которых отсутствуют в списке категорий

In [111]:
df_amo_missing_prefix = df_amo_temp.query('prefix in @missing_prefix')

In [112]:
df_amo_missing_prefix.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,prefix_V
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395,P1G
20,24438053,Успешно_реализовано,12345.0,2023-01-06 10:45:05,2023-02-06 19:37:00,450.0,NaN,Подарочный_сертификат_на_12_345_рублей_(Бумажн...,12795.0,12345.0,12345,Универсальный_Мосвейк_12345,W&F_260561,NaN,NaN,NaN,NaN,NaN,NaT,,WF,260561,WF
26,24445523,Успешно_реализовано,8450.0,2023-01-06 16:56:58,2023-02-06 19:37:00,450.0,NaN,Мотокросс_и_Эндуро_(Варианты:_Персональная_тре...,8450.0,8000.0,8450,Мото_Мотокросс,MT2_164020,NaN,NaN,NaN,NaN,Погашен,2023-08-16,,MT2,164020,MT2
65,23462101,Успешно_реализовано,5450.0,2023-01-13 16:31:03,2023-01-14 15:40:00,450.0,NaN,Фридайвинг_(Варианты:_Индивидуальная_тренировк...,5450.0,5000.0,5450,Фридайвинг,F1_272111,NaN,NaN,NaN,NaN,NaN,NaT,,F1,272111,F1
81,23500055,Успешно_реализовано,21800.0,2023-01-18 16:44:39,2023-01-18 18:43:00,NaN,NaN,Прыжок_с_парашютом_(Варианты:_Прыжок_с_инструк...,21800.0,21800.0,21800,Парашюты,SP267323,NaN,NaN,NaN,NaN,Погашен,2023-01-25,,,,


In [113]:
df_amo_missing_prefix.shape

(150, 23)

Количество записей для префиксов, отсутствующих в общей базе

In [114]:
df_amo_missing_prefix.groupby(['prefix_V'])['prefix_V'].count()

prefix_V
          32
'+         1
DT230      1
DT240      2
F1         3
GR1X1      1
MF         2
MT2       24
P1        12
P1G        8
P1P        5
P2         4
P2G        3
PS         1
S         10
S2         4
SC         1
SC7        2
SKIN60     1
SP         9
SP2        3
ST         2
WF        17
WS30T      1
WVS30      1
Name: prefix_V, dtype: int64

In [115]:
df111 = df_amo_temp.query('prefix == "DT230"')

In [116]:
df111.head(10)

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,prefix_V
2463,25061353,Успешно_реализовано,9000.0,2023-11-08 19:07:04,2023-12-08 14:29:00,NaN,NaN,Заезд_на_Дрифт_Трайке_(Варианты:_Заезд_30_мину...,9000.0,9000.0,9000,Мото_Дрифт-трайк,DT230_288316,NaN,NaN,NaN,NaN,NaN,NaT,,DT230,288316,DT230


Префиксу WF соответствует W&F

In [117]:
df_amo_temp.loc[df_amo_temp['prefix_V'] == 'WF','prefix_V'] = 'W&F'

In [118]:
df_amo_temp.loc[df_amo_temp['prefix'] == 'WF','prefix'] = 'W&F'

In [119]:
df_amo_temp.loc[df_amo_temp['prefix'] == 'VWF','prefix'] = 'VW&F'

#### База АМО без prefix

In [120]:
df_amo_without_prefix = df_amo_temp.loc[df_amo_temp['prefix'] == '']

In [121]:
df_amo_without_prefix.shape

(32, 23)

In [122]:
df_amo_without_prefix.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,prefix_V
81,23500055,Успешно_реализовано,21800.0,2023-01-18 16:44:39,2023-01-18 18:43:00,NaN,NaN,Прыжок_с_парашютом_(Варианты:_Прыжок_с_инструк...,21800.0,21800.0,21800,Парашюты,SP267323,NaN,NaN,NaN,NaN,Погашен,2023-01-25,,,,
85,23503361,Успешно_реализовано,12795.0,2023-01-19 10:45:43,2023-01-19 10:59:00,450.0,NaN,Универсальный_12345_(Бумажный_или_электронный:...,12795.0,12345.0,12795,Универсальный_Мосвейк_12345,W&F284959,NaN,NaN,NaN,NaN,NaN,NaT,,,,
105,23547341,Успешно_реализовано,22250.0,2023-01-25 00:23:05,2023-01-25 17:57:00,450.0,NaN,Прыжок_с_парашютом_(Варианты:_Прыжок_с_инструк...,22250.0,21800.0,22500,Парашюты,SP291000,NaN,NaN,NaN,NaN,NaN,NaT,,,,
223,23672219,Успешно_реализовано,4500.0,2023-02-13 18:39:27,2023-02-14 09:45:00,NaN,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4500.0,4500.0,4500,Параплан,P1G238229,NaN,NaN,NaN,NaN,NaN,NaT,,,,
239,23691835,Успешно_реализовано,5000.0,2023-02-16 20:33:34,2023-02-17 23:16:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,5000,Параплан,80688,NaN,NaN,NaN,NaN,NaN,NaT,,,,


In [123]:
#df_amo_temp = df_amo_temp.loc[df_amo_temp['prefix'] != '']

In [124]:
df_amo_temp['card_type'] ='paper'

In [125]:
#df_amo_temp['series'] =''

In [126]:
df_amo_temp['design_type'] = 'design1'

In [127]:
df_amo_temp['status'] = ''

In [128]:
#df_amo_temp['stock_status'] = ''

In [129]:
df_amo_temp['sale_price'] = 0

In [130]:
df_amo_temp['expiry_date'] = ''

In [131]:
df_amo_temp['created_at'] = ''

In [132]:
df_amo_temp['updated_at'] = ''

In [133]:
df_amo_temp['deleted_at'] = ''

In [134]:
df_amo_temp.loc[(df_amo_temp['card_used_date'].notna()) |  (df_amo_temp['статус_погашения'] == 'Погашен'),'status'] = 'sold_used'

In [135]:
df_amo_temp['supplier_payment_date'] = df_amo_temp['card_used_date']

In [136]:
df_amo_temp.query('status=="sold_used"').head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,prefix_V,card_type,design_type,status,sale_price,expiry_date,created_at,updated_at,deleted_at,supplier_payment_date
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395,P1G,paper,design1,sold_used,0,,,,,2023-09-09
7,23770797,Успешно_реализовано,0.0,2023-01-03 16:05:22,2023-01-03 16:06:00,NaN,NaN,NaN,NaN,NaN,NaN,Фридайвинг,FR_R2666164,#23769895,NaN,NaN,NaN,Погашен,2023-10-07,,FR,R2666164,FR,paper,design1,sold_used,0,,,,,2023-10-07
11,23985509,Успешно_реализовано,14000.0,2023-01-04 12:26:13,2023-03-04 11:50:00,700.0,NaN,Вертолет_Robinson_R44_(Варианты:_15_минут_полё...,13200.0,12500.0,13200,Авиа_Вертолет,RB15_166049,NaN,NaN,NaN,NaN,Погашен,2023-08-25,,RB15,166049,RB15,paper,design1,sold_used,0,,,,,2023-08-25
14,23988163,Успешно_реализовано,2600.0,2023-01-04 20:35:41,2023-01-04 20:43:00,0.0,NaN,Катание_на_картинге_(Варианты:_20_минут_катани...,2600.0,2600.0,2600,Картинг,VKA2_2890897,NaN,NaN,NaN,NaN,Погашен,2023-02-04,,VKA2,2890897,KA2,paper,design1,sold_used,0,,,,,2023-02-04
15,24191536,Успешно_реализовано,6000.0,2023-01-05 01:03:54,2023-01-05 15:49:00,0.0,NaN,Вейк-сёрфинг_за_катером_(Варианты:_30_минут_Ma...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,VWS30T_2987448,NaN,NaN,NaN,NaN,Погашен,2023-07-22,,VWS30T,2987448,WS30T,paper,design1,sold_used,0,,,,,2023-07-22


### Обработка поля **status**

In [137]:
df_amo_temp.loc[(df_amo_temp['card_used_date'].notna()) |  \
                (df_amo_temp['статус_погашения'] == 'Погашен'),'status'] = 'sold_used'

### Обработка **sale_price**

#### Строки, являющиеся корневыми сделками

In [138]:
lst_root_deal = list(df_amo_temp.query('корневая_сделка.notna()')['корневая_сделка'])

In [139]:
print(lst_root_deal)

['#23769895', '#24693067', '#24964891', '#23459729', '1715029001', '#23586703', '#23514709', '#24696037', '#24696037', '#24975785', '#23681147', '#23681147', '#23705733', '#23703937', '#23710155', '#23716727', '#23716727', '#23716727', '23717485', '23718601', '23718601', '23718601', '#23724197', '#23723181', '23727459', '#23728823', '#23729565', '#23729565', '#23729565', '23732757', '#23720975', '#23748665', '#23756947', '#24204699', '#24715547', '#25885733', '#23906061', '#23918309', '#23918309', '23923847', '23923847', '#23931737', '23963847', '#23968933', '#23965909', '#23772175', '24214499', '24723003', '24992009', '#25228541', '#25538207', '#25538207', '#24072407', '#24083997', '#24098553', '#24103527', '24108151', '#24122097', 'КR2_230212', '#24177302', '#23787591', '#24996157', '24292405', '#24294657', '#24308743', '24349847', '24378991', '#24386001', '24402725', '24403171', '24403285', '24432999', '24434571', '24436793', '24437073', '#23624295', '#23624295', '#23624295', '24740

In [140]:
len(lst_root_deal)

135

In [141]:
def clear_num(num):
    """
    Очистка id корневой сделки и перевод ео в int
    """
    if num.isdigit():
        return int(num)
    else:
        l = len(num)-1
        while l>0:
            if num[l].isdigit():
                l -= 1
            else:
                break
        return int(num[l+1:])

Проверка:

In [142]:
print(clear_num('КR2_230212'))

230212


In [143]:
lst_root_deal = list(map(clear_num,lst_root_deal))

Список id строк, являющимися корневыми сделками

In [144]:
print(lst_root_deal)

[23769895, 24693067, 24964891, 23459729, 1715029001, 23586703, 23514709, 24696037, 24696037, 24975785, 23681147, 23681147, 23705733, 23703937, 23710155, 23716727, 23716727, 23716727, 23717485, 23718601, 23718601, 23718601, 23724197, 23723181, 23727459, 23728823, 23729565, 23729565, 23729565, 23732757, 23720975, 23748665, 23756947, 24204699, 24715547, 25885733, 23906061, 23918309, 23918309, 23923847, 23923847, 23931737, 23963847, 23968933, 23965909, 23772175, 24214499, 24723003, 24992009, 25228541, 25538207, 25538207, 24072407, 24083997, 24098553, 24103527, 24108151, 24122097, 230212, 24177302, 23787591, 24996157, 24292405, 24294657, 24308743, 24349847, 24378991, 24386001, 24402725, 24403171, 24403285, 24432999, 24434571, 24436793, 24437073, 23624295, 23624295, 23624295, 24740295, 24548805, 24579543, 24579543, 24579543, 24579543, 24586559, 24606011, 24621605, 24646105, 24651727, 24669423, 24676465, 24678867, 24029505, 24030407, 25010665, 25010665, 25010665, 25011851, 25022279, 24815607,

In [145]:
df_amo_temp.query('amo_id in @lst_root_deal').head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,prefix_V,card_type,design_type,status,sale_price,expiry_date,created_at,updated_at,deleted_at,supplier_payment_date
8,23772175,Успешно_реализовано,9350.0,2023-01-03 17:59:38,2023-04-03 17:35:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,9350.0,9000.0,9350,Квадроциклы,KV1X1_162569,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,162569,KV1X1,paper,design1,,0,,,,,NaT
40,24693067,Успешно_реализовано,8000.0,2023-01-07 18:27:21,2023-01-07 18:44:00,0.0,NaN,"Роупджампинг_(Варианты:_Одиночный_прыжок,_Бума...",8000.0,8000.0,8000,Роупджампинг,VRJ_3466463,NaN,NaN,NaN,NaN,NaN,NaT,,VRJ,3466463,RJ,paper,design1,,0,,,,,NaT
61,23459729,Успешно_реализовано,11000.0,2023-01-13 11:16:07,2023-01-13 11:44:00,NaN,NaN,Искусственная_волна_Flow_Moscow_(Варианты:_1_ч...,11000.0,11000.0,11000,Искусственная_волна,VCW1_2484684,NaN,NaN,NaN,NaN,Погашен,2023-06-29,,VCW1,2484684,CW1,paper,design1,sold_used,0,,,,,2023-06-29
131,23514709,Успешно_реализовано,19900.0,2023-01-29 22:13:18,2023-01-31 01:27:00,NaN,NaN,Школа_дрифта_(Бумажный_или_электронный:_Электр...,19900.0,19900.0,19900,Авто_Дрифт,VDRC2H_2534706,NaN,NaN,NaN,NaN,Погашен,2023-09-02,,VDRC2H,2534706,DRC2H,paper,design1,sold_used,0,,,,,2023-09-02
133,23586703,Успешно_реализовано,16450.0,2023-01-31 11:09:39,2023-01-31 17:52:00,450.0,NaN,Мотокросс_и_Эндуро_(Варианты:_Персональная_тре...,16450.0,16000.0,16450,Мото_Мотокросс,MT2_184809,NaN,NaN,NaN,NaN,Погашен,2023-03-05,,MT2,184809,MT2,paper,design1,sold_used,0,,,,,2023-03-05


Если в сделке 1 товар и сделка не имеет производных сделок, то использовать значение поля SUBTOTAL

#### Расчет sale_price для обычных товаров

In [146]:
df_amo_temp.loc[(df_amo_temp['номер_2-го_сертификата'].isna()) & \
                (df_amo_temp['сертификат_на_тренера'].isna()) &\
                (df_amo_temp['сертификат_на_фотографа'].isna()) &\
                (~df_amo_temp['amo_id'].isin(lst_root_deal)) &\
                (df_amo_temp['корневая_сделка'].isna()),\
                'sale_price'] = df_amo_temp['subtotal']

Проверка

Сумма поля sale_price для сделок, являющихся корневыми

In [147]:
df_amo_temp.query('amo_id in @lst_root_deal')['sale_price'].sum()

0.0

Сумма поля sale_price для сделок, не являющихся корневыми, не имеющих 2 серт, не имеющих серт на тренера и фотографа

In [148]:
df_amo_temp.query('amo_id in @lst_root_deal')['subtotal'].sum()

1360439.0

In [149]:
df_amo_temp.head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,prefix_V,card_type,design_type,status,sale_price,expiry_date,created_at,updated_at,deleted_at,supplier_payment_date
0,23580385,Успешно_реализовано,5200.0,2023-01-02 09:55:23,2023-05-02 21:14:00,NaN,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213421,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213421,KA4,paper,design1,,5200.0,,,,,NaT
1,23594083,Успешно_реализовано,4850.0,2023-01-02 11:40:07,2023-03-02 01:20:00,350.0,NaN,Квадроциклы_(Варианты:_1_квадрик_1_час_без_пас...,4850.0,4500.0,4850,Квадроциклы,KV1X1_125315,NaN,NaN,NaN,NaN,NaN,NaT,,KV1X1,125315,KV1X1,paper,design1,,4500.0,,,,,NaT
2,23596587,Успешно_реализовано,4850.0,2023-01-02 16:02:03,2023-02-02 14:53:00,350.0,NaN,Полет_на_параплане_(Варианты:_1_полет_с_инстру...,4850.0,4500.0,4850,Параплан,P1G_259395,NaN,NaN,NaN,NaN,Погашен,2023-09-09,,P1G,259395,P1G,paper,design1,sold_used,4500.0,,,,,2023-09-09
3,23596239,Успешно_реализовано,6000.0,2023-01-02 16:04:38,2023-02-02 14:53:00,NaN,NaN,Вейк_за_катером_(Варианты:_30_минут_Mastercraf...,6000.0,6000.0,6000,Вейксёрф_и_вейк_за_катером,WS30_239328,NaN,NaN,NaN,TR1_210177,NaN,NaT,,WS30,239328,WS30,paper,design1,,0.0,,,,,NaT
4,23598009,Успешно_реализовано,5200.0,2023-01-02 19:57:22,2023-02-02 14:54:00,0.0,NaN,катание_на_картинге_(Варианты:_40_минут_катани...,5200.0,5200.0,5200,Картинг,KA4_213697,NaN,NaN,NaN,NaN,NaN,NaT,,KA4,213697,KA4,paper,design1,,5200.0,,,,,NaT


База АМО c prefix

In [150]:
df_amo_temp = df_amo_temp.loc[df_amo_temp['prefix'] != '']

In [151]:
df_amo_temp.shape

(2639, 32)

In [152]:
df_amo_temp['card_type'] = df_amo_temp['prefix'].apply(lambda x: 'electronic' if x[0] == 'V' else 'paper')

In [153]:
df_amo_temp.query('amo_id==26139623').head()

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,prefix_V,card_type,design_type,status,sale_price,expiry_date,created_at,updated_at,deleted_at,supplier_payment_date
2543,26139623,Успешно_реализовано,11450.0,2023-11-28 13:06:38,2023-02-12 22:29:00,450.0,NaN,"Снегоходы_(Варианты:_1_снежик_на_1_час,_Бумажн...",11450.0,11000.0,11450,Снегоходы,SN1X_199622,NaN,SN1X_191480,NaN,NaN,NaN,NaT,,SN1X,199622,SN1X,paper,design1,,0.0,,,,,NaT
2667,26139623,NaN,NaN,2023-11-28 13:06:38,2023-02-12 22:29:00,NaN,NaN,NaN,NaN,NaN,NaN,,SN1X_191480,NaN,NaN,NaN,NaN,NaN,NaT,,SN1X,191480,SN1X,paper,design1,,0.0,,,,,NaT


In [154]:
#df_amo_temp.query('корневая_сделка.notna()')

#Проверка:

In [155]:
#pp = df_amo_temp.query('amo_id == 26139797')

In [156]:
#pp

#### Расчет sale_price для корневых, имеющих 2-й сертификат

Функция для определения sale_price для 

In [157]:
def insert_sale111(row):    
    v = row['sale_price']
    if row['sale_price'] == 0:
        amo_id = row['amo_id']
        pr = row['prefix']
        v = 0
        str1 = f'sale_price != 0 & prefix == "{pr}"'
        try:
            v = df_amo_temp.query(str1).iloc[-1]['sale_price']
        except:
            pass
        finally:
            return v
    return v

In [158]:
def insert_sale(row):    
    v = row['sale_price']
    #print(v)
    if row['sale_price'] == 0:
        d = row['дата_создания_сделки']
        amo_id = row['amo_id']
        pr = row['prefix']
        #v = 0
        str1 = f'sale_price != 0 & prefix == "{pr}"'
        try:
            #v = df_amo_temp.query(str1).iloc[-1]['sale_price']
            arr = df_amo_temp.query(str1)    
            arr['diff_date'] = arr['дата_создания_сделки'] - d
            arr = arr.sort_values(by='diff_date',ascending=True)
            arr.reset_index(drop= True , inplace= True)
            v = arr.iloc[0]['sale_price']
        except:
            pass
        finally:
            return v
    return v

In [159]:
def insert_saleV(row):    
    v = row['sale_price']
    #print(v)
    if row['sale_price'] == 0:
        d = row['дата_создания_сделки']
        amo_id = row['amo_id']
        pr = row['prefix_V']
        #v = 0
        str1 = f'sale_price != 0 & prefix_V == "{pr}"'
        try:
            #v = df_amo_temp.query(str1).iloc[-1]['sale_price']
            arr = df_amo_temp.query(str1)    
            arr['diff_date'] = arr['дата_создания_сделки'] - d
            arr = arr.sort_values(by='diff_date',ascending=True)
            arr.reset_index(drop= True , inplace= True)
            v = arr.iloc[0]['sale_price']
        except:
            pass
        finally:
            return v
    return v

In [160]:
df_amo_temp['sale_price'] = df_amo_temp.apply(insert_sale,axis=1)

In [161]:
df_amo_temp['sale_price'] = df_amo_temp.apply(insert_saleV,axis=1)

Проверка:

In [162]:
pp = df_amo_temp.query('amo_id == 26139623')

In [163]:
pp

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,prefix_V,card_type,design_type,status,sale_price,expiry_date,created_at,updated_at,deleted_at,supplier_payment_date
2543,26139623,Успешно_реализовано,11450.0,2023-11-28 13:06:38,2023-02-12 22:29:00,450.0,NaN,"Снегоходы_(Варианты:_1_снежик_на_1_час,_Бумажн...",11450.0,11000.0,11450,Снегоходы,SN1X_199622,NaN,SN1X_191480,NaN,NaN,NaN,NaT,,SN1X,199622,SN1X,paper,design1,,5000.0,,,,,NaT
2667,26139623,NaN,NaN,2023-11-28 13:06:38,2023-02-12 22:29:00,NaN,NaN,NaN,NaN,NaN,NaN,,SN1X_191480,NaN,NaN,NaN,NaN,NaN,NaT,,SN1X,191480,SN1X,paper,design1,,5000.0,,,,,NaT


sale_price появился

#### Строки, оставшиеся без sale_price

In [164]:
dd0 = df_amo_temp.query('sale_price == 0').head(10)

In [165]:
dd0.shape

(10, 32)

In [166]:
dd0.head(10)

,amo_id,этап_сделки,бюджет,дата_создания_сделки,sale_date,delivery_price,стоимость_доставки,products,price,subtotal,выручка_от_клиента,направление_(вид_сертификата),code,корневая_сделка,номер_2-го_сертификата,сертификат_на_фотографа,сертификат_на_тренера,статус_погашения,card_used_date,id,prefix,number,prefix_V,card_type,design_type,status,sale_price,expiry_date,created_at,updated_at,deleted_at,supplier_payment_date
249,23703937,Успешно_реализовано,10500.0,2023-02-19 10:08:55,2023-02-20 09:58:00,NaN,NaN,"Снегоходы_(Варианты:_1_снежик_на_1_час,_Бумажн...",10000.0,10000.0,10500,Снегоходы,SN1X1_116859,NaN,NaN,NaN,NaN,Погашен,2023-03-03,,SN1X1,116859,SN1X1,paper,design1,sold_used,0.0,,,,,2023-03-03
254,23706073,Успешно_реализовано,0.0,2023-02-19 15:26:55,2023-02-20 09:58:00,NaN,NaN,NaN,NaN,NaN,NaN,Снегоходы,SN1X1_193747,#23703937,NaN,NaN,NaN,Погашен,2023-03-03,,SN1X1,193747,SN1X1,paper,design1,sold_used,0.0,,,,,2023-03-03
371,23732921,Успешно_реализовано,0.0,2023-02-24 00:42:33,2023-02-24 00:42:00,NaN,NaN,NaN,NaN,NaN,NaN,Мото_Дрифт-трайк,VDT62_2660855,23732757,NaN,NaN,NaN,NaN,NaT,,VDT62,2660855,DT62,electronic,design1,,0.0,,,,,NaT
682,24088395,Успешно_реализовано,12400.0,2023-04-16 20:02:43,2023-04-16 20:16:00,NaN,NaN,Подарочный_сертификатна_водные_лыжи_(Выберите_...,12400.0,12400.0,12400,Водные_лыжи,SKIN60_156264,NaN,NaN,NaN,TR212535,NaN,NaT,,SKIN60,156264,SKIN60,paper,design1,,0.0,,,,,NaT
843,24996969,Успешно_реализовано,24277.0,2023-05-08 15:44:41,2023-10-08 18:14:00,0.0,NaN,Комната_ярости_(Варианты:_Сезонное_обострение_...,24277.0,24277.0,24277,Квест_Комната_Ярости,VYAS_3897866,NaN,NaN,NaN,NaN,NaN,NaT,,VYAS,3897866,YAS,electronic,design1,,0.0,,,,,NaT
1089,24434571,Успешно_реализовано,13400.0,2023-05-31 13:08:13,2023-05-31 13:32:00,0.0,NaN,"Роупджампинг_(Варианты:_Прыжок_в_тандеме,_Бума...",13400.0,13400.0,13400,Роупджампинг,VRJ2X_3134392,NaN,NaN,NaN,NaN,Погашен,2023-06-14,,VRJ2X,3134392,RJ2X,electronic,design1,sold_used,0.0,,,,,2023-06-14
1290,24596539,Успешно_реализовано,6800.0,2023-06-19 17:34:39,2023-06-21 23:01:00,NaN,NaN,Подарочный_сертификатна_вейк_за_катером_(Выбер...,6800.0,6800.0,6800,Вейксёрф_и_вейк_за_катером,ES130_164358,NaN,VWS30T_3324255DIS,NaN,NaN,NaN,NaT,,ES130,164358,ES130,paper,design1,,0.0,,,,,NaT
1323,24614375,Успешно_реализовано,7300.0,2023-06-21 17:57:54,2023-06-24 10:12:00,0.0,NaN,Подарочный_сертификатна_вейк_за_катером_(Выбер...,6800.0,6800.0,7300,Вейксёрф_и_вейк_за_катером,ES130_112723,NaN,NaN,NaN,TR2_220282,Погашен,2023-08-07,,ES130,112723,ES130,paper,design1,sold_used,0.0,,,,,2023-08-07
1911,25078171,Успешно_реализовано,16000.0,2023-08-14 17:08:31,2023-08-14 19:41:00,NaN,NaN,Полет_на_параплане_(Варианты:_Курс_«Первый_сам...,16000.0,16000.0,16000,Катер_без_капитана_(Летолодка),VLL3H_4147449,NaN,VPPT_3929836DIS,NaN,NaN,NaN,NaT,,VLL3H,4147449,LL3H,electronic,design1,,0.0,,,,,NaT
1922,25084647,Успешно_реализовано,0.0,2023-08-15 14:26:13,2023-08-15 14:32:00,NaN,NaN,NaN,NaN,NaN,NaN,Квест_Комната_Ярости,VYAS_3946815,25084249,NaN,NaN,NaN,NaN,NaT,,VYAS,3946815,YAS,electronic,design1,,0.0,,,,,NaT


Несколько записей так и не получили sale_price

In [167]:
df_amo_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2639 entries, 0 to 2670
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   amo_id                         2639 non-null   int64         
 1   этап_сделки                    2590 non-null   object        
 2   бюджет                         2590 non-null   float64       
 3   дата_создания_сделки           2639 non-null   datetime64[ns]
 4   sale_date                      2639 non-null   datetime64[ns]
 5   delivery_price                 1631 non-null   float64       
 6   стоимость_доставки             9 non-null      float64       
 7   products                       2425 non-null   object        
 8   price                          2425 non-null   float64       
 9   subtotal                       2425 non-null   float64       
 10  выручка_от_клиента             2471 non-null   object        
 11  направление_(вид_

## Запись в файлы

### Объединение датафреймов без префиксов и с отсутствующими префиксами и запись в файл

In [168]:
df_amo_missing_prefix = df_amo_temp.query('prefix in @missing_prefix')

In [169]:
df_amo_missing_prefix.shape

(101, 32)

In [170]:
frames = [df_amo_without_prefix, df_amo_missing_prefix]

In [171]:
cert_w_missing_prefix = pd.concat(frames)

In [172]:
cert_w_missing_prefix.shape

(133, 32)

In [173]:
cert_w_missing_prefix.to_csv('cert_w_missing_prefix.csv')

### Запись отсутствующих в Списке подкатегорий prefix в файл

In [174]:
df_missing_prefix = pd.DataFrame({'prefix':missing_prefix})

In [175]:
df_missing_prefix.shape

(41, 1)

In [176]:
df_missing_prefix.head()

,prefix
0,
1,'+
2,DT230
3,DT240
4,EFL


In [177]:
df_missing_prefix.to_csv('missing_prefix.csv')

### (п.10) Запись товаров, необходимые поля которого заполнены

In [178]:
df_amo_temp['amo_lead_id'] = df_amo_temp['amo_id']

In [179]:
df_amo_temp['amo_id'] = ''

Итоговые столбцы из АМО

In [180]:
lst_afm = ['id', 
           'amo_id',
           'amo_lead_id',
           'prefix',
           'prefix_V',
           'code',
           'card_type',
           'design_type',
           'status',
           'supplier_payment_date',
           'sale_price',
           'sale_date',
           'expiry_date',
           'card_used_date',
           'created_at',
           'updated_at',
           'deleted_at',
           'number',
]

Итоговые столбцы из Реестра

In [181]:
df_values['prefix_V'] = df_values['prefix']

In [182]:
lst_val = ['activity_value_id','prefix_V']

Датафреймы для объединения

In [183]:
df_amo_for_merge = df_amo_temp.query('prefix_V in @prefix_values')

In [184]:
df_amo_for_merge = df_amo_for_merge[lst_afm]

In [185]:
df_amo_for_merge.head()

,id,amo_id,amo_lead_id,prefix,prefix_V,code,card_type,design_type,status,supplier_payment_date,sale_price,sale_date,expiry_date,card_used_date,created_at,updated_at,deleted_at,number
0,,,23580385,KA4,KA4,KA4_213421,paper,design1,,NaT,5200.0,2023-05-02 21:14:00,,NaT,,,,213421
1,,,23594083,KV1X1,KV1X1,KV1X1_125315,paper,design1,,NaT,4500.0,2023-03-02 01:20:00,,NaT,,,,125315
3,,,23596239,WS30,WS30,WS30_239328,paper,design1,,NaT,4500.0,2023-02-02 14:53:00,,NaT,,,,239328
4,,,23598009,KA4,KA4,KA4_213697,paper,design1,,NaT,5200.0,2023-02-02 14:54:00,,NaT,,,,213697
5,,,23766769,DT40,DT40,DT40_285464,paper,design1,,NaT,6000.0,2023-02-03 20:30:00,,NaT,,,,285464


In [186]:
df_value_for_merge = df_values[lst_val]

Объединение датафреймов

In [187]:
df_list = pd.merge(df_amo_for_merge, df_value_for_merge,how="inner",on='prefix_V')

In [188]:
df_list.head()

,id,amo_id,amo_lead_id,prefix,prefix_V,code,card_type,design_type,status,supplier_payment_date,sale_price,sale_date,expiry_date,card_used_date,created_at,updated_at,deleted_at,number,activity_value_id
0,,,23580385,KA4,KA4,KA4_213421,paper,design1,,NaT,5200.0,2023-05-02 21:14:00,,NaT,,,,213421,105
1,,,23598009,KA4,KA4,KA4_213697,paper,design1,,NaT,5200.0,2023-02-02 14:54:00,,NaT,,,,213697,105
2,,,23987233,KA4,KA4,KA4_278259,paper,design1,,NaT,5200.0,2023-01-04 17:40:00,,NaT,,,,278259,105
3,,,24692227,VKA4,KA4,VKA4_3419464,electronic,design1,,NaT,5500.0,2023-01-07 15:11:00,,NaT,,,,3419464,105
4,,,24694159,KA4,KA4,KA4_264053,paper,design1,,NaT,5500.0,2023-04-07 16:06:00,,NaT,,,,264053,105


In [189]:
df_list = df_list.drop('prefix_V', axis=1)

In [190]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2045 entries, 0 to 2044
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     2045 non-null   object        
 1   amo_id                 2045 non-null   object        
 2   amo_lead_id            2045 non-null   int64         
 3   prefix                 2045 non-null   object        
 4   code                   2045 non-null   object        
 5   card_type              2045 non-null   object        
 6   design_type            2045 non-null   object        
 7   status                 2045 non-null   object        
 8   supplier_payment_date  778 non-null    datetime64[ns]
 9   sale_price             1962 non-null   float64       
 10  sale_date              2045 non-null   datetime64[ns]
 11  expiry_date            2045 non-null   object        
 12  card_used_date         778 non-null    datetime64[ns]
 13  cre

Запись в файл

In [191]:
df_list.to_csv('certs_to_import.csv')

### (п.11) Список товаров, чьи префиксы не обнаружены в list_of_values.csv, но есть в codes_archieve

In [192]:
df_amo_temp = df_amo_temp.query('prefix_V in @archive_prefix')

In [193]:
df_amo_for_write = df_amo_temp[lst_afm]

In [194]:
df_amo_for_write = df_amo_for_write.drop('prefix_V', axis=1)

In [195]:
df_amo_for_write.head()

,id,amo_id,amo_lead_id,prefix,code,card_type,design_type,status,supplier_payment_date,sale_price,sale_date,expiry_date,card_used_date,created_at,updated_at,deleted_at,number
42,,,24693703,VWS20,VWS20_3415971,electronic,design1,sold_used,2023-10-09,3600.0,2023-01-07 23:00:00,,2023-10-09,,,,3415971
44,,,24962283,EF30,EF30_173522,paper,design1,,NaT,6200.0,2023-01-08 12:01:00,,NaT,,,,173522
55,,,25212865,VWS20,VWS20_4035259,electronic,design1,,NaT,3600.0,2023-01-09 11:29:00,,NaT,,,,4035259
94,,,23521391,VWS20,VWS20_2431777,electronic,design1,sold_used,2023-07-20,3000.0,2023-01-21 20:26:00,,2023-07-20,,,,2431777
139,,,23601589,WS60,WS60_267335,paper,design1,,NaT,NaN,2023-04-02 15:04:00,,NaT,,,,267335


In [196]:
df_amo_for_write.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 42 to 2668
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     226 non-null    object        
 1   amo_id                 226 non-null    object        
 2   amo_lead_id            226 non-null    int64         
 3   prefix                 226 non-null    object        
 4   code                   226 non-null    object        
 5   card_type              226 non-null    object        
 6   design_type            226 non-null    object        
 7   status                 226 non-null    object        
 8   supplier_payment_date  93 non-null     datetime64[ns]
 9   sale_price             179 non-null    float64       
 10  sale_date              226 non-null    datetime64[ns]
 11  expiry_date            226 non-null    object        
 12  card_used_date         93 non-null     datetime64[ns]
 13  cre

In [197]:
df_amo_for_write.to_csv('certs_to_import_archieve_prefix.csv')